# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [2]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [10]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [26]:
# print names of all countries
for child in document_tree.getroot():
    print(child.tag, child.attrib)

country {'memberships': 'org-BSEC org-CEI org-CD org-SELEC org-CE org-EAPC org-EBRD org-EITI org-FAO org-IPU org-IAEA org-IBRD org-ICC org-ICAO org-ICCt org-Interpol org-IDA org-IFRCS org-IFC org-IFAD org-ILO org-IMO org-IMF org-IOC org-IOM org-ISO org-OIF org-ITU org-ITUC org-IDB org-MIGA org-NATO org-OSCE org-OPCW org-OAS org-OIC org-PCA org-UN org-UNCTAD org-UNESCO org-UNIDO org-UPU org-WCO org-WFTU org-WHO org-WIPO org-WMO org-UNWTO org-WTO', 'capital': 'cty-Albania-Tirane', 'area': '28750', 'car_code': 'AL'}
country {'memberships': 'org-AG org-BIS org-BSEC org-CD org-SELEC org-CE org-EMU org-EAPC org-EBRD org-ECB org-EIB org-CERN org-ESA org-EU org-FATF org-FAO org-IGAD org-IPU org-IAEA org-IBRD org-ICC org-ICAO org-ICJ org-ICCt org-Interpol org-IDA org-IEA org-IFRCS org-IFC org-IFAD org-IHO org-ILO org-IMO org-IMSO org-IMF org-IOC org-IOM org-OIF org-ITSO org-ITU org-ITUC org-MIGA org-NATO org-NEA org-NSG org-OECD org-OSCE org-OPCW org-OAS org-PCA org-UN org-UNCTAD org-UNESCO org

In [19]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print('* ' + element.find('name').text + ':'),
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print(capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

# 10 Countries with Lowest Infant Mortality Rates

In [22]:
import pandas as pd
document_tree = ET.parse('./data/mondial_database.xml')

In [25]:
# set mortality_rates dictionary
mortality_rates = {}

# loop through tree to find countries and infant mortality rates
for element in document_tree.iterfind('country'):
    country = element.find('name')
    infant_mortality = element.find('infant_mortality')
    
    # append keys and values to dictionary
    if country != None and infant_mortality != None:
        mortality_rates[country.text] = float(infant_mortality.text)

# create dataframe from newly created dictionary, keys as rows
mortality_rates_df = pd.DataFrame.from_dict(mortality_rates, orient='index')

# label column and print out lowest 10 rates
mortality_rates_df.columns=['Infant_Mortality_Rate']
mortality_rates_df.sort_values(by='Infant_Mortality_Rate', ascending=True).head(10)



,Infant_Mortality_Rate
Monaco,1.81
Japan,2.13
Norway,2.48
Bermuda,2.48
Singapore,2.53
Sweden,2.60
Czech Republic,2.63
Hong Kong,2.73
Macao,3.13
Iceland,3.15


# 10 Cities with Largest Population

In [43]:
# set city popultion dictionary
city_pops = {}

# Loop to find city names and provinces
for country in document_tree.iterfind('country'):
    cities = country.findall('city')
    provinces = country.findall('province')

# add province population to cities
    for province in provinces:
        cities += province.findall('city')

# Loops for city name and last population entry
    for city in cities:
        city_name = city.find('name').text
        population = city.find('.//population[last()]')
        
# Skips popluation count if node is empty
        if population != None:
            city_pop = int(population.text)
        # add city and pop to dictionary
        city_pops[city_name] = city_pop

# create dataframe from newly created dictionary, keys as rows
city_pops_df = pd.DataFrame.from_dict(city_pops, orient='index')

# label column and print out 10 highest populations
city_pops_df.columns=['Population']
city_pops_df.sort_values(by='Population', ascending=False).head(10)



,Population
Shanghai,22315474
Istanbul,13710512
Mumbai,12442373
Moskva,11979529
Beijing,11716620
São Paulo,11152344
Tianjin,11090314
Guangzhou,11071424
Delhi,11034555
Shenzhen,10358381


# 10 Ethnic Groups With the Largest Overall Populations (sum of best/latest estimates over all countries)

In [65]:
# set country population dictionary
country_pops = {}

# Loops to find country name and last population count
for element in document_tree.iterfind('country'):
    country = element.find('name')
    population = element.find('.//population[last()]')
    
    # append population values to dictionary
    if country != None and population != None:
        country_pops[country.text] = int(population.text)

# create dataframe from dictionary, keys as rows
country_pops_df = pd.DataFrame.from_dict(country_pops, orient='index')

# label columns
country_pops_df.reset_index(drop=False, inplace=True)
country_pops_df.columns = ['Country', 'Population']
country_pops_df.head()
        

,Country,Population
0,Puerto Rico,3725789
1,Colombia,47661787
2,Ceuta,82376
3,United Kingdom,64105654
4,Honduras,8045990


In [95]:
# set ethnicity list and column labels for future dataframe
ethnic_groups = []
cols = ['Country', 'Ethnic_group', 'Percentage']

# Loop through document_tree to find ethnic groups
for country in document_tree.iterfind('country'):
    countries = country.find('name').text
    
    for ethnic_group in country.iterfind('ethnicgroup'):
        ethnicity = ethnic_group.text
        percentage = ethnic_group.attrib['percentage']
        ethnic_groups.append([countries, ethnicity, percentage])
        



In [96]:
# Create dataframe and combine with country_pops_df
ethnic_groups_df = pd.DataFrame(ethnic_groups, columns=cols)
ethnic_groups_df = ethnic_groups_df.merge(country_pops_df, on='Country')
ethnic_groups_df.head()

,Country,Ethnic_group,Percentage,Population
0,Albania,Albanian,95,2800138
1,Albania,Greek,3,2800138
2,Greece,Greek,93,10816286
3,Macedonia,Macedonian,64.2,2059794
4,Macedonia,Albanian,25.2,2059794


In [99]:
#Rename columns
ethnic_groups_df = ethnic_groups_df.rename(columns={'Population': 'Country_Population'})

#Create ethnic_group_population column
ethnic_groups_df['Ethnic_group_Population'] = (ethnic_groups_df['Percentage'].astype(float) / 100) * ethnic_groups_df['Country_Population']
ethnic_groups_df['Ethnic_group_Population'] = ethnic_groups_df['Ethnic_group_Population'].astype(int)
ethnic_groups_df.head()

,Country,Ethnic_group,Percentage,Country_Population,Ethnic_group_Population
0,Albania,Albanian,95,2800138,2660131
1,Albania,Greek,3,2800138,84004
2,Greece,Greek,93,10816286,10059145
3,Macedonia,Macedonian,64.2,2059794,1322387
4,Macedonia,Albanian,25.2,2059794,519068


In [100]:
# Display top 10 ethnicities using groupby
ethnic_groups_final = ethnic_groups_df.groupby('Ethnic_group').sum()
ethnic_groups_final.sort_values(by='Ethnic_group_Population', ascending=False, inplace=True)
ethnic_groups_final.head(10)

,Country_Population,Ethnic_group_Population
Ethnic_group,,
Han Chinese,1360720000,1245058800
Indo-Aryan,1210854977,871815583
European,1157295639,494872201
African,975352746,318325104
Dravidian,1210854977,302713744
Mestizo,279743964,157734349
Bengali,149772364,146776916
Russian,322438406,131856989
Japanese,127298000,126534212


# Name and Country of Longest River

In [150]:
# create country dictionary for dataframe
country_dict = {}

# Adds country code ('car_code') as key to dictionary with value as country name
for code in document_tree.iterfind('country'):
    country_dict[code.attrib['car_code']] = code.find('name').text

# create dataframe, values as rows
country_df = pd.DataFrame.from_dict(country_dict, orient='index')
country_df.head()

,0
B,Belgium
WEST,West Bank
ARU,Aruba
KIR,Kiribati
WS,Samoa


In [151]:
# set column names
country_df.reset_index(drop=False, inplace=True)
country_df.columns = ['Country Code', 'Country']
country_df.head()

,Country Code,Country
0,B,Belgium
1,WEST,West Bank
2,ARU,Aruba
3,KIR,Kiribati
4,WS,Samoa


In [167]:
# create river list
river_list = []

# loop for river names, river codes, river lengths
for rivers in document_tree.iterfind('river'):
    river_name = rivers.find('name').text
    river_length_temp = rivers.find('length')
    if river_length_temp != None:
        river_length = river_length_temp.text

# separate rivers that run through different countries
        for river_code in rivers.attrib['country'].split():
            river_country_code = country_dict[river_code]
    river_list.append([river_country_code, river_code, river_name, float(river_length)])

In [168]:
# create river dataframe and print longest river
river_labels = ['Country', 'Code', 'River_Name', 'River_Length']
river_df = pd.DataFrame(river_list, columns=river_labels)
river_df.sort_values(by='River_Length', ascending=False).head(1)

# Longest River

,Country,Code,River_Name,River_Length
174,Peru,PE,Amazonas,6448.0


# Name and Country of Largest Lake

In [184]:
# create lake list
lake_list = []

# loop for lake names, lake codes, lake areas
for lakes in document_tree.iterfind('lake'):
    lake_name = lakes.find('name').text
    lake_area_temp = lakes.find('area')
    if lake_area_temp != None:
        lake_area = lake_area_temp.text

        # separate lakes by country
        for lake_code in lakes.attrib['country'].split():
            lake_country_code = country_dict[lake_code]
    lake_list.append([lake_country_code, lake_code, lake_name, float(lake_area)])

In [185]:
# create lake dataframe and print largest lake
lake_labels = ['Country', 'Code', 'Lake_Name', 'Lake_Area']
lake_df = pd.DataFrame(lake_list, columns=lake_labels)
lake_df.sort_values(by='Lake_Area', ascending=False).head(1)

#largest lake

,Country,Code,Lake_Name,Lake_Area
54,Turkmenistan,TM,Caspian Sea,386400.0


# Name and Country of Highest Airport

In [197]:
# create airport list
airport_list = []

# loop for airport names, airport codes, airport elevations
for airports in document_tree.iterfind('airport'):
    airport_name = airports.find('name').text
    airport_elevation_temp = airports.find('elevation')
    if airport_elevation_temp != None:
        airport_elevation = airport_elevation_temp.text
        
        # separate airport by country
        for airport_code in airports.attrib['country'].split():
            airport_country_code = country_dict[airport_code]
    airport_list.append([airport_country_code, airport_code, airport_name, airport_elevation])      


In [202]:
# create airport df and print highest airport
airport_labels = ['Country', 'Code', 'Aiport_Name', 'Airport_Elevation']
airport_temp_df = pd.DataFrame(airport_list, columns=airport_labels)

# convert values to numeric, some are NoneType
airport_df = airport_temp_df.apply(pd.to_numeric, errors='ignore')
airport_df.head()
airport_df.sort_values(by='Airport_Elevation', ascending=False).head(1)

# highest airport

,Country,Code,Aiport_Name,Airport_Elevation
80,Bolivia,BOL,El Alto Intl,4063.0
